In [ ]:
import requests
import datetime
import time
import os, sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "."))
from config.filters import SPESE, INCOME, RIMBORSI, INVESTIMENTI, BUONI_PASTO
from dotenv import load_dotenv
load_dotenv()
WALLET_API_TOKEN = os.getenv("WALLET_API_TOKEN")

In [2]:
def call_wallet_api(endpoint: str, args: dict = None, base_url = "https://rest.budgetbakers.com/wallet"):
    headers = {"Authorization": f"Bearer {WALLET_API_TOKEN}"}
    request_url = f"{base_url}/{endpoint}"
    # send the request with a payload of params
    response = requests.get(request_url, headers=headers, params=args)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API call failed with status code {response.status_code}: {response.text}")

accounts = call_wallet_api("v1/api/accounts")
accounts = {account['name']: account['id'] for account in accounts['accounts']}

In [3]:
accounts

{'Prepaid XME plus': '1a5c2400-70b0-421f-8bc4-39f95bf879f1',
 'Trade Republic': '24df231a-1cc3-4bcb-8528-242c4889bc1c',
 'Buoni pasto': '2b044f45-7a57-4095-bbad-9cd2f49a7fa0',
 'Fineco': '355bbb52-ecb7-4717-b170-9eeb2cd0b063',
 'Intesa San Paolo': '3b544b6c-9653-4968-ac71-daa82252b738',
 'Banco BPM': '67ea39ef-ac84-48ac-9c48-ff8f9049a1c1',
 'Scalable Capital': '6e418d8c-ab2b-4977-abc1-f1ce7a864506',
 'Cash': '7296cfee-ff1a-497f-8382-fcc7a265e23a',
 'PayPal': '8cd5ee39-e099-47e7-90e1-03585a866e7b',
 'Revolut': '8ff2d76e-f475-4af6-b265-f677b2c1996a',
 'Satispay': 'cf067959-f56a-4cb0-bd6d-ace82640d3b4'}

In [19]:
first_day_of_this_month = datetime.date.today().replace(day=1)
first_day_of_last_month = (first_day_of_this_month - datetime.timedelta(days=1)).replace(day=1).isoformat()
last_date_of_last_month = (first_day_of_this_month - datetime.timedelta(days=1)).isoformat()
# first_day_of_last_month = (datetime.date.today() - datetime.timedelta(days=1)).isoformat()
# last_date_of_last_month = (datetime.date.today()).isoformat()
# first_day_of_last_month = "2025-01-01"
# last_date_of_last_month = "2025-12-31"
print(f"Fetching records from {first_day_of_last_month} to {last_date_of_last_month}")

records_df = pd.DataFrame()

i = 0

for account_name, account_id in accounts.items():
    while True:    
        n_records_retrieved = i * 100

        args = {
                "accountId": account_id,
                "recordDate": f"lte.{last_date_of_last_month}",
                "limit": 100,
                "offset": n_records_retrieved
            }
        time.sleep(1)
        print(f"Fetching records for account {account_name} with offset {n_records_retrieved}")
        records = call_wallet_api("v1/api/records", args=args)

        if len(records['records']) > 0:
            account_records = records['records']
            for record in account_records:
                cat_id = record['category']['id']
                cat_name = record['category']['name']
                date = record['recordDate']
                amount = record['amount']["value"]
                label_ids = [label['id'] for label in record.get('labels', [])]
                label_names = [label['name'] for label in record.get('labels', [])]

                new_row = {
                    "account": account_name,
                    "amount": amount,
                    "date": date,
                    "category_id": cat_id,
                    "category_name": cat_name,
                    "label_ids": label_ids,
                    "label_names": label_names
                }
                records_df = pd.concat([records_df, pd.DataFrame([new_row])], ignore_index=True)
                
            i += 1
        else:
            i = 0
            break

records_df = records_df[records_df['date'] >= first_day_of_last_month]

Fetching records from 2026-01-01 to 2026-01-31
Fetching records for account Prepaid XME plus with offset 0
Fetching records for account Trade Republic with offset 0
Fetching records for account Trade Republic with offset 100
Fetching records for account Trade Republic with offset 200
Fetching records for account Trade Republic with offset 300
Fetching records for account Buoni pasto with offset 0
Fetching records for account Fineco with offset 0
Fetching records for account Intesa San Paolo with offset 0
Fetching records for account Intesa San Paolo with offset 100
Fetching records for account Banco BPM with offset 0
Fetching records for account Banco BPM with offset 100
Fetching records for account Scalable Capital with offset 0
Fetching records for account Scalable Capital with offset 100
Fetching records for account Cash with offset 0
Fetching records for account Cash with offset 100
Fetching records for account PayPal with offset 0
Fetching records for account PayPal with offset 10

In [20]:
records_df

,account,amount,date,category_id,category_name,label_ids,label_names
158,Trade Republic,-26.80,2026-01-01T06:21:11Z,7c7dfaee-3be0-474e-b337-bd8b4e735940,Eventi,[8921b28b-b4ac-406d-9f84-4bf07dddc81f],[Dublin]
159,Trade Republic,4.26,2026-01-01T09:32:59Z,7af26741-a212-47da-a50c-baeb717e9db4,Interessi,[],[]
160,Trade Republic,-19.89,2026-01-01T12:00:00Z,68b6ced6-1c86-4534-8c88-704e0de6519e,,[],[]
161,Trade Republic,-90.00,2026-01-01T16:01:00Z,fe2cd110-9275-4f28-92f0-ecb3bfb1663e,Ristoranti & Fast-food,[8921b28b-b4ac-406d-9f84-4bf07dddc81f],[Dublin]
162,Trade Republic,-9.95,2026-01-01T18:00:38Z,d69edfb5-81ef-42b5-8908-49db1a17e9a6,Regali,[8921b28b-b4ac-406d-9f84-4bf07dddc81f],[Dublin]
...,...,...,...,...,...,...,...
468,Revolut,13.16,2026-01-31T10:05:42Z,c196e895-83d8-4118-b744-94321501287e,Rimborsi,[],[]
469,Satispay,5.00,2026-01-13T09:52:16Z,c196e895-83d8-4118-b744-94321501287e,Rimborsi,[],[]
470,Satispay,4.80,2026-01-13T13:19:09Z,b6a83884-8a6b-4071-a660-e1a36bc28cde,"Trasferimento, Prelievo",[],[]
471,Satispay,-7.60,2026-01-13T16:53:21Z,a0e0eaa6-9f83-43fd-afd7-59e20d0381ae,Shopping,[],[]


In [21]:
records_df.shape

(144, 7)

In [22]:
spese_df = records_df[records_df['category_id'].isin(SPESE['categories'].keys()) &
           (records_df['amount'] < 0)]
spese_amount = round(spese_df.amount.sum(), 2)

In [23]:
income_df = records_df[records_df['category_id'].isin(INCOME['categories'].keys()) &
           (records_df['amount'] > 0)]
income_amount = round(income_df.amount.sum(), 2)

In [24]:
rimborsi_df = records_df[records_df['category_id'].isin(RIMBORSI['categories'].keys()) &
           (records_df['amount'] > 0)]
rimborsi_amount = round(rimborsi_df.amount.sum(), 2)

In [25]:
investimenti_df = records_df[records_df['category_id'].isin(INVESTIMENTI['categories'].keys()) &
           (records_df['amount'] > 0)]
investimenti_amount = round(investimenti_df.amount.sum(), 2)

In [26]:
buoni_pasto_df = records_df[records_df['category_id'].isin(BUONI_PASTO['categories'].keys()) &
           (records_df['amount'] > 0)]
buoni_pasto_amount = round(buoni_pasto_df.amount.sum(), 2)

In [30]:
cash_flow = round(income_amount + rimborsi_amount + spese_amount, 2)
print(f"Spese: {spese_amount}€")
print(f"Income: {income_amount}€")
print(f"Rimborsi: {rimborsi_amount}€")
print("\n")
print(f"Buoni pasto: {buoni_pasto_amount}€")
print(f"Cash flow: {cash_flow}€")
print(f"Investimenti: {investimenti_amount}€")

Spese: -2764.17€
Income: 2072.6€
Rimborsi: 464.51€


Buoni pasto: 0.0€
Cash flow: -227.06€
Investimenti: 4.26€


Categories non mappate

In [ ]:
records_df[records_df['category_id'].isin([
    'ea566b98-446f-40eb-8071-f5289acf6716',
    #  'dcd71755-22e5-4f87-baa7-b07ad5cbbcb8',
     'd259f3dc-0630-4845-ad3b-af4facc22e0d',
     'a6daf078-746f-4de8-8d53-ddeab5abfe83',
    #  '0285055c-755c-4008-a5ef-84a8f2e77f0b',
    #  '68b6ced6-1c86-4534-8c88-704e0de6519e', # Tempo libero
     'dcd71755-22e5-4f87-baa7-b07ad5cbbcb8'
     ]
                                          )].sort_values('date')

,account,amount,date,category_id,category_name,label_ids,label_names


### Categories

In [4]:
categories = call_wallet_api("v1/api/categories")

In [11]:
categories

{'categories': [{'id': '025dcc04-511a-48a7-9d54-9c8330ec7fb3',
   'color': '#64DD17',
   'name': 'Bellezza & Benessere',
   'createdAt': '2019-12-23T14:06:57Z',
   'updatedAt': '2026-02-23T08:07:44Z',
   'customCategory': False,
   'customColor': False,
   'customName': False},
  {'id': '0285055c-755c-4008-a5ef-84a8f2e77f0b',
   'color': '#cccccc',
   'name': '',
   'createdAt': '2020-03-04T10:27:18Z',
   'updatedAt': '2026-02-23T08:07:44Z',
   'customCategory': False,
   'customName': False},
  {'id': '03e0aa81-0318-46b0-9b63-91d7420e7ef6',
   'color': '#AB47BC',
   'name': 'Pedaggio',
   'createdAt': '2021-05-20T20:32:53Z',
   'updatedAt': '2026-02-23T08:07:44Z',
   'customCategory': False,
   'customColor': True,
   'customName': True,
   'iconName': 'car-filled'},
  {'id': '043f8b6a-60de-4ec7-bfb7-4c808f738c66',
   'color': '#4fC3F7',
   'name': 'Abbigliamento & Scarpe',
   'createdAt': '2019-12-22T14:16:58Z',
   'updatedAt': '2026-02-23T08:07:44Z',
   'customCategory': False,
   '

In [5]:
categories_df = pd.DataFrame()

i = 0
while True:
    n_cat_retrieved = i * 100
    args = {
        "limit": 100,
        "offset": n_cat_retrieved
    }
    cat_page = call_wallet_api("v1/api/categories", args=args)
    
    if not cat_page['categories']:  # Stop when no more items are returned
        break
    
    for cat in cat_page['categories']:
        id = cat['id']
        name = cat['name']
        categories_df = pd.concat([categories_df, pd.DataFrame([{"id": id, "name": name}])], ignore_index=True)
    
    i += 1

In [6]:
for i, row in categories_df.iterrows():
    print(row['name'], row['id'])

Bellezza & Benessere 025dcc04-511a-48a7-9d54-9c8330ec7fb3
 0285055c-755c-4008-a5ef-84a8f2e77f0b
Pedaggio 03e0aa81-0318-46b0-9b63-91d7420e7ef6
Abbigliamento & Scarpe 043f8b6a-60de-4ec7-bfb7-4c808f738c66
Automatic bank statements reading 05a4d998-706f-4167-a964-f15a7a4a2767
Spesa 06b45615-a7d0-4f00-8992-95d6f9907c55
Drugstore 0e0ae827-900d-466b-b91c-b5af11f6d5c8
TV & Streaming 11ad0c36-f9cc-4ed7-b82c-601780c46491
Veicoli 19bff33c-875e-4ce7-bf16-64ff29ebf59c
Manutenzione veicoli 1c796611-0d25-4611-b143-a827f02e309f
Risparmi 1fe10154-42a3-42b4-8ec1-b88980ce0346
Lotterie & Gioco d'azzardo 202089f9-0aa1-4959-9dd9-7e905a2a99a6
Internet 22abfec7-d7b9-402b-896a-2242142b75fe
Culture, sport events 2409b680-1dfb-4926-99c8-652a92dc5a6a
Fuel 26426aa6-55ef-43e2-afef-a0285fd5a424
Health & beauty 2979979c-6fa4-44f2-8a45-958749697ad0
Taxes 3179b457-1ad6-4b0a-ae54-005ac04f6055
Jewels, accessories 32818c0a-5472-4ddf-905c-66ee1f68dee0
Altro 35794129-8e21-43bb-aba3-e09b53ec53f6
Trasporti 36d3a3a7-6395-49f8-

# Filters